In [11]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal
import pandas as pd

### ouvrir un signal et obtenir sa transformée de Fourier normalisée et en Log10

In [ ]:
def transfourier(chemin):
    signal_dict = np.load(str(chemin)) # signal_dict est un dictionnaire
    #contenant les données contenues dans le fichier signal.npz
    s = signal_dict['s'] # extraction du signal utile
    t = signal_dict['t'] # extraction du vecteur temps associé au signal

    px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
    title=f"signal {chemin}").show()

    nu, S = san.trans_fourier(s, t) #on fait la transformée de Fourier
    S = S - np.mean(S) #on enlève la composante continue de la transformée
    S = np.absolute(S) #on fait le module

    fondamentale = (san.detect_fondamentale(S, nu, 0.7*np.max(S))) #on relève la fondamentale tant que ça marche

    S = np.log10(S) #passage en échelle Log10
    S = S / np.max(S) #normalisation de l'amplitude

    px.line(x=nu, y=S, labels={"x":"Temps (HZ)", "y":"Amplitude"},
    title=f"TF {chemin}").show()

    return(nu, S, fondamentale)

#transfourier("apprentissage/a_1.npz")

### Trouver fondamentales et H-premières harmoniques

In [ ]:
def obtenir_pics(lettre, rang_signal, nu, S, fondamentale, H, DATA_vect_Freq, DATA_vect_Amp):
    #contient le vecteur amplitude associé à chaque spectre 1 à 5

    freqs_harmoniques = [fondamentale]
    for var_rang_harmonique in range (H): #ajout des harmoniques suivantes
        freqs_harmoniques.append((var_rang_harmonique+1)*fondamentale)
        
    SIGNAL_Harmonique_Freq, SIGNAL_Harmonique_Amp= san.detect_pics(S,freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)
    DATA_vect_Freq[lettre][rang_signal] = SIGNAL_Harmonique_Freq
    DATA_vect_Amp[lettre][rang_signal] = SIGNAL_Harmonique_Amp
    return(DATA_vect_Freq, DATA_vect_Amp)

### moyenne de signaux

In [ ]:
def moyenne(lettre, nbre_signaux, H, DATA_vect_Freq, DATA_vect_Amp, DATA_moyennes, nu):
    temp_moy_freq, temp_moy_amp = 0, 0
    for var_rang_harmonique2 in range(H+1):
        for var_rang_signal in range(nbre_signaux):
            temp_moy_freq += DATA_vect_Freq[lettre][var_rang_signal][var_rang_harmonique2]
            temp_moy_amp += DATA_vect_Amp[lettre][var_rang_signal][var_rang_harmonique2]
        
    DATA_moyennes[lettre][0] += 1/5*temp_moy_freq
    DATA_moyennes[lettre][1] += 1/5*temp_moy_amp

    """Tracés de la courbe des signaux de la lettre étudiée"""
    COURBE_moyenne = px.line(x=DATA_moyennes[lettre][0], y=DATA_moyennes[lettre][1], labels={"x":"", "y":"Amplitude"},
    title=lettre, markers='markers', range_x=[0, 5000], range_y=[-1.15,1.15]).show() #trace du vecteur moyen
    

### Calcul de distance du signal ETUDIÉ avec la base d'apprentisssage

In [ ]:
def distance(ETUDE_Harmonique_Amp, lettre, H, DATA_moyennes):
    ecartAmp = {"a": 0,"e":0,"i":0,"o":0,"u":0} #contient des listes avec les ecarts de chaque pic avec la moyenne
    ecarAmin = 10 
    Juste_Lettre = "0"

    for var_lettre in enumerate(lettres) : #on parcoure chaque lettre
        temp_Amp = []
        for var_rang_harmonique3 in range (H + 1): #on parcoure les harmoniques du signal
            temp_Amp.append(int(np.abs(ETUDE_Harmonique_Amp[j] - DATA_moyennes[var_lettre][1][var_rang_harmonique3]) * 1000))
        
        for var_rang_harmonique4 in range(H + 1): #on reduit le nombre de décimales
            temp_Amp[var_rang_harmonique4] = temp_Amp[var_rang_harmonique4]/1000

        ecartAmp[var_lettre] = temp_Amp
       
        ecartAMoyen = np.mean(ecartAmp[var_lettre])
        if ecartAMoyen < ecarAmin:
            ecarAmin = ecartAMoyen
            Juste_Lettre = var_lettre
    return(Juste_Lettre)

### automatiser la base de données

In [ ]:
def baseDeDonnees(lettres, nbre_signaux,H, DATA_vect_Freq, DATA_vect_Amp):
    temp_Vects_lettre = []
    for i in enumerate(lettres):
        for var_rang_signal in range(nbre_signaux):
            nu, S, fondamentale = transfourier(f"etude/{i}_{j}")
            obtenir_pics(lettres, var_rang_signal, nu, S, fondamentale, H, DATA_vect_Freq, DATA_vect_Amp)


### paramètres d'étude et exécuter l'analyse de spectre

In [ ]:
LETTRES = ["a","e","i","o","u"] #sons étudiés
H = 13  #nombre d'harmoniques
nbre_signaux = 5 #nombre de signaux par lettre dans la base de donnée d'apprentissage


DATA_vect_Freq = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur frequence associé à chaque spectre 1 à 5
DATA_vect_Amp = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur amplitude associé à chaque spectre 1 à 5
DATA_moyennes = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]} #[liste des frequences moyennes, liste des amplitudes moyennes]